In [1]:
# URL from which pdfs to be downloaded
url = "https://usrbrands.com/products/c19r-29-single-solid-door-narrow-depth-reach-in-refrigerator?_pos=1&_sid=c851f596b&_ss=r"

In [2]:
manufacturer = 'Coldline'
sku          = 'C12R'
prod_type    = 'Reach-In Refrigerator'
cost         = 1415
family       = 'Refrigeration' # Equipment

In [3]:
import requests
from bs4 import BeautifulSoup
# Requests URL and get response object
response = requests.get(url)
# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')
desc_txt = soup.find(class_='product-meta__description rte').text
desc_txt = desc_txt.replace('\n','')
desc_txt = desc_txt.strip()

# get Description field
Description = '<h4>The ' + manufacturer + ' ' + sku + ' ' + prod_type + '</h4>\n<p>' + desc_txt + '</p>'
print(Description)

<h4>The Coldline C12R Reach-In Refrigerator</h4>
<p>The narrow 25" depth stainless steel reach-in commercial refrigerator is perfect for small restaurants, delis, or bakeries, cafes. With a spacious cabinet and adjustable shelves, the unit can still fit in narrow hallways, upstairs, or small kitchens. The 1/5 HP compressor provides plenty of power for hot kitchens. The bottom mount raises the bottom shelf for easier access. With lockable wheels, door locks, digital temperature control, and coated corrosion resistant shelves, it has all of the features of more expensive commercial units at a reasonable price.</p>


In [4]:
# download all pdfs
import requests
from bs4 import BeautifulSoup
# Requests URL and get response object
response = requests.get(url)
# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')
# Find all hyperlinks present on webpage
links = soup.find_all('a')
i = 0
# From all links check for pdf link and
# if present download file
for link in links:
    if ('.pdf' in link.get('href', [])):
        i += 1
        print("Downloading file: ", i)
  
        # Get response object for link
        response = requests.get(link.get('href'))
        url_link = link.get('href')
        
        # Write content in pdf file
        pdf = open("pdf"+str(i)+".pdf", 'wb')
        pdf.write(response.content)
        pdf.close()
        break
        print("File ", i, " downloaded")
for link in links:
    if ('.jpg' in link.get('href', [])):
        img1_link = link.get('href')
        break
print("All PDF files downloaded")

All PDF files downloaded


In [5]:
# get features
import PyPDF4
import re
file_path = 'pdf1.pdf'
#file_path = 'c19r-specsheet.pdf'

with open(file_path, mode='rb') as f:
    reader = PyPDF4.PdfFileReader(f)
    page = reader.getPage(0)
    txt = page.extractText()
f.close() 
cleanString = re.sub('[^A-Za-z0-9 \n]+', '', txt)
new_txt = re.split('\n',cleanString)
new_txt = new_txt[3:-17]
new_txt2 = [x for x in new_txt if len(x)>1]

# convert in required html format
features = '<ul>\n'
for i in range(len(new_txt2)):
    features += '  <li>' + new_txt2[i] + '</li>\n'
features += '</ul>'

# write it to a file
#f = open('new_file.txt','w')
#f.write(features)
#f.close()

# print features
features

'<ul>\n  <li>Stainless steel exterior and interior</li>\n  <li>Aluminum back</li>\n  <li>Narrow Depth</li>\n  <li>Refrigeration system keeps food at safe temperatures</li>\n  <li>Selfclosing doors with 90 degree stayopen feature</li>\n  <li>Locks and keys included</li>\n  <li>Adjustable heavyduty PVC coated shelves</li>\n  <li>Recessed door handle</li>\n  <li>Electronic control system</li>\n  <li>Preinstalled heavyduty casters with brakes</li>\n  <li>Removable installation board </li>\n  <li>HydroCarbon R290 refrigerant</li>\n</ul>'

In [6]:
title_txt = re.split('\n',txt)
title = title_txt[0]
title = title.replace('ﬂ','\"')
title = manufacturer + ' ' + title
print(title)

Coldline C19R 29" Single Solid Door Narrow Depth Reach-In Refrigerator


In [7]:
# Read tables
import tabula
import pandas as pd
file_path = 'pdf1.pdf'

# area [top,left,bottom,right]
read_pdf_dfs = tabula.read_pdf(file_path, pages=2, area=[10,0,750,315])[0]
read_pdf_dfs.columns = ['col1','col2']
# read_pdf_dfs

for i in range(2,len(read_pdf_dfs)):
    if pd.isna(read_pdf_dfs.col2[i]):
        df1 = pd.DataFrame(read_pdf_dfs.iloc[:i,])
        df1.dropna(inplace=True)
        df2 = pd.DataFrame(read_pdf_dfs.iloc[i:,])
        df2.dropna(inplace=True)
        break
    else:
        pass

Got stderr: Dec 30, 2021 12:19:00 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JPEG2000 image: Java Advanced Imaging (JAI) Image I/O Tools are not installed



In [8]:
# convert Dimensions in required html format
Dimensions = '<table>\n<tbody>\n'
#print(Dimensions)
for i in range(len(df1)):
    #print('index = ',i)
    Dimensions += '<tr>\n<td>'
    Dimensions += df1.iloc[i:i+1,0].to_string(header=False, index=False) + '</td>\n'
    #print(Dimensions)
    Dimensions += '<td>' + df1.iloc[i:i+1,1].to_string(header=False, index=False) + '</td>'
    #print(Dimensions)
    Dimensions += '\n</tr>\n'
    #print(Dimensions)
    
Dimensions += '</tbody>\n</table>'
print(Dimensions)

<table>
<tbody>
<tr>
<td>Exterior Dimensions</td>
<td>29”W x 25.4”D x 81.5”H</td>
</tr>
<tr>
<td>Interior Dimensions</td>
<td>24.4”W x 20.4”D x 60.3”H</td>
</tr>
<tr>
<td>Packaging Dimensions</td>
<td>30.3”W x 27.5”D x 85.5”H</td>
</tr>
<tr>
<td>Net Volume</td>
<td>19 cu. ft.</td>
</tr>
<tr>
<td>Unit Weight</td>
<td>282 lb.</td>
</tr>
<tr>
<td>Shipping Weight</td>
<td>308 lb.</td>
</tr>
</tbody>
</table>


In [9]:
# Width and Cubic Feet
if family=='Refrigeration':
    width = df1.col2.iloc[0].split('W')[0]
    CubicFeet = df1.col2[df1.col1=='Net Volume'].values[0]
else:
    width = ''
    CubicFeet = ''

In [10]:
# convert Specifications in required html format
Specifications = '<table>\n<tbody>\n'
#print(Dimensions)
for i in range(len(df2)):
    #print('index = ',i)
    Specifications += '<tr>\n<td>'
    Specifications += df2.iloc[i:i+1,0].to_string(header=False, index=False) + '</td>\n'
    #print(Dimensions)
    Specifications += '<td>' + df2.iloc[i:i+1,1].to_string(header=False, index=False) + '</td>'
    #print(Dimensions)
    Specifications += '\n</tr>\n'
    #print(Dimensions)
    
Specifications += '</tbody>\n</table>'
print(Specifications)

<table>
<tbody>
<tr>
<td>Voltage</td>
<td>115</td>
</tr>
<tr>
<td>Hertz</td>
<td>60</td>
</tr>
<tr>
<td>Phase</td>
<td>1</td>
</tr>
<tr>
<td>Amps</td>
<td>2.8</td>
</tr>
<tr>
<td>Plug Type</td>
<td>NEMA 5-15P</td>
</tr>
<tr>
<td>Power Cord Length</td>
<td>96”</td>
</tr>
<tr>
<td>Temperature Range</td>
<td>33°F to 41°F</td>
</tr>
<tr>
<td>Refrigerant</td>
<td>R290</td>
</tr>
<tr>
<td>Max. Ambient Temp</td>
<td>90°F</td>
</tr>
<tr>
<td>Defrosting</td>
<td>Automatic</td>
</tr>
<tr>
<td>Temperature Controller</td>
<td>Digital</td>
</tr>
<tr>
<td>Horsepower</td>
<td>1/5</td>
</tr>
<tr>
<td>Exterior Material</td>
<td>430  Stainless Steel</td>
</tr>
<tr>
<td>Interior Material</td>
<td>430 Stainless Steel</td>
</tr>
<tr>
<td>Insulation Material</td>
<td>Foamed-in-Place Polyurethane</td>
</tr>
<tr>
<td>Shelf Size</td>
<td>3 Qty: 23.4”W x 19.4”D</td>
</tr>
<tr>
<td>Shelf Material</td>
<td>Epoxy Coated Wire</td>
</tr>
<tr>
<td>Shelf Load Capacity</td>
<td>90 lb.</td>
</tr>
<tr>
<td>Adjustable She

In [12]:
# store all info in a dataframe
my_df = pd.DataFrame(columns = ['manufacturer','SKU','Cost','markup','ListPrice','Kitchenall price','Title','Product Type',
                                'Family','Description','Features','Dimensions','Certifications','Specifications',
                                'productImage1','productImage2','productImage3','productImage4','productImage5','Video 1',
                                'Spec Sheet','Manual','Width','Cubic Feet'])
my_df.loc[len(my_df)] = [manufacturer,sku,cost,1.3*cost,'','',title,prod_type,family,Description,features,Dimensions,
                         'cETLus, ETLsan, Plug: NEMA 5-15P',Specifications,img1_link,'','','','','',url_link,'',width,CubicFeet]

excel_df = pd.read_excel('outputfile.xlsx')
excel_df = excel_df.append(my_df, ignore_index=True)
excel_df.to_excel('outputfile.xlsx',index=False)

# extract pdf sketches on page 2

In [13]:
# convert 2nd pdf page to image

# require poppler (download, extract and mention path into variable poppler_path) 
from pdf2image import convert_from_path
images = convert_from_path("pdf1.pdf", 500,poppler_path=r'C:\Users\Sanghavi\Desktop\upwork\Abe soskil, The Better Group LLC\poppler-0.68.0\bin')
images[1].save('pdf_full_pg2.png', "PNG") # need only 2nd page to save

# other solution (don't have control on image quality)
#import fitz
## read the page
#pdffile = "pdf1.pdf"
#doc = fitz.open(pdffile)
#page = doc.loadPage(1)  # number of page
#pix = page.get_pixmap()
#output = "new_pic.png"
#pix.save(output)

In [14]:
# split image to get right side part

import cv2   
# Read the image
img = cv2.imread('pdf_full_pg2.png')
print(img.shape)
height = img.shape[0]
width = img.shape[1]
# Cut the image in half
width_cutoff = int(width / 2)
s1 = img[:, :width_cutoff]
s2 = img[int(height//6.5):int(height//1.07), width_cutoff:]
#cv2.imwrite("new_pic1.png", s1)
cv2.imwrite("pdf_sketch.png", s2)

(5500, 4250, 3)


True